In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import numpy
import matplotlib.pyplot
import pathlib

# Setup paths

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
initial_path = pathlib.Path(r'initial_data')
destination_path = pathlib.Path(r'catchment_trimmed_data')

boundary_name = pathlib.Path(r'boundary\boundary.shp')
coast_name = pathlib.Path(r'north_island\north_island.shp')
background_dem_name = pathlib.Path(r'background_dem\NZDEM_SoS_v1-0_16_Wellington_gf.tif')
background_bathymetry_name = pathlib.Path(r'background_bathymetry\sounding-points-hydro-122k-190k.shp')

destination_dem_name = pathlib.Path(r'dem.nc')
destination_bathymetry_name = pathlib.Path(r'offshore_bathymetry')
destination_boundary_name = pathlib.Path(r'boundary')
destination_boundary_land_name = pathlib.Path(r'boundary_land')

# Load in data

In [ ]:
boundary = geopandas.read_file(base_path/initial_path/boundary_name)
north_island = geopandas.read_file(base_path/initial_path/coast_name)
bathymetry = geopandas.read_file(base_path/initial_path/background_bathymetry_name)
dem = rioxarray.rioxarray.open_rasterio(base_path/initial_path/background_dem_name, masked=True)

# Set to same projection

In [ ]:
crs = 2193
boundary = boundary.to_crs(crs)
north_island = north_island.to_crs(crs)
bathymetry = bathymetry.to_crs(crs)
dem.rio.set_crs(crs);

# View boundary with background data to trim

In [ ]:
dem_shape = geopandas.GeoSeries([shapely.geometry.Polygon([(dem.x.data.max(), dem.y.data.max()), (dem.x.data.min(), dem.y.data.max()),
                                        (dem.x.data.min(), dem.y.data.min()), (dem.x.data.max(), dem.y.data.min())])])

In [ ]:
dem_shape.loc[0]

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 10))
gs = f.add_gridspec(1, 2)

ax1 = f.add_subplot(gs[0, 0]) 
dem_shape.plot(ax=ax1, color='teal', edgecolor='gray', linewidth=2)
north_island.plot(ax=ax1, color='None', edgecolor='magenta', linewidth=2)
bathymetry.plot(ax=ax3, marker='o', color='red')
boundary.plot(ax=ax2, color='None', edgecolor='cyan', linewidth=2)
f.colorbar(p1, ax=ax1)

ax2 = f.add_subplot(gs[0, 1]) 
p2 = ax2.pcolormesh(dem.x[:], dem.y[:], dem.values[0], cmap='viridis', shading='auto')
bathymetry.plot(ax=ax2, marker='o', color='red', markersize=10)
north_island.plot(ax=ax2, color='None', edgecolor='magenta', linewidth=3)
boundary.plot(ax=ax2, color='None', edgecolor='cyan', linewidth=3)
f.colorbar(p4, ax=ax2)
matplotlib.pyplot.xlim([1.765e6, 1.78e6])
matplotlib.pyplot.ylim([5.466e6, 5.4755e6])

ax1.set_title("Background DEM outline, coast, bathymetry and catchment boundary")
ax4.set_title("Combined")

# Trim background data to boundary

In [ ]:
boundary_land = geopandas.clip(boundary, north_island)
bathymetry_trimmed = geopandas.clip(bathymetry, boundary)
dem_trimmed = dem.rio.clip(boundary_land.geometry)

# Plot trimmed data

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

p = ax.pcolormesh(dem_trimmed.x[:],dem_trimmed.y[:],dem_trimmed.values[0],cmap='viridis', shading='auto')
boundary_land.plot(ax=ax, color='None', edgecolor='lime', linewidth=5)
bathymetry_trimmed.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.colorbar(p)

ax.set_title("Combined")

# Save out trimmed data

In [ ]:
boundary.to_file(base_path/destination_path/destination_boundary_name)
boundary_land.to_file(base_path/destination_path/destination_boundary_land_name)
bathymetry_trimmed.to_file(base_path/destination_path/destination_bathymetry_name)
dem_trimmed.to_netcdf(base_path/destination_path/destination_dem_name)